In [1]:
import pandas as pd
import altair as alt

data = pd.read_csv("cereal.csv")

In [35]:
brush = alt.selection_interval(resolve='global')
single = alt.selection_point(on='mouseover', nearest=True)

concat_plot = alt.Chart(data).mark_circle().encode(
    alt.Y('calories',scale=alt.Scale(domain=[0, 175])).title('Calories per serving'),
    size = alt.Size('cups', scale=alt.Scale(range=(0,200)),legend = alt.Legend(title="Cups per Serving",orient='bottom')),
    tooltip = ['name','calories','sugars','carbo','protein','fat', 'cups'],
    color = alt.condition(brush, 'child', alt.ColorValue('gray'),legend = alt.Legend(title="Target Demographic",orient='bottom')),
    opacity = alt.condition(brush, alt.value(0.75), alt.value(0.1))
).properties(
    height=200,
    width=200
)

detail = alt.Chart(data, title=alt.TitleParams('Interactive Cereal Evaluation Plot - Select Cereals from Below Plots', anchor='middle')).mark_bar().encode(
    x = alt.X('name',axis=alt.Axis(labelAngle=-45,),title="Name"),
    y = alt.Y('calories',title="Calories",scale=alt.Scale(domain=[0, 175])),
    color='child',
    tooltip = ['name','calories','sugars','carbo','protein','fat', 'cups'],
    opacity = alt.condition(brush, alt.value(0.75), alt.value(0.1))
# ).add_params( # Altair is smart enough to not let you select and filter the same plot
#     brush    
# ).transform_filter(brush
).properties(
    height=200,
    width=1000)

small_multiples = alt.hconcat(concat_plot.encode(x=alt.X('sugars', scale=alt.Scale(domain=[-1,16])).title('Sugar per Serving (g)')),
                              concat_plot.encode(x=alt.X('carbo', scale=alt.Scale(domain=[-1,25])).title('Carbs per Serving (g)')),
                              concat_plot.encode(x=alt.X('fat', scale=alt.Scale(domain=[-1,6])).title('Fat per Serving (g)')),
                              concat_plot.encode(x=alt.X('protein',scale=alt.Scale(domain=[-1,7])).title('Protein per Serving (g)')) 
                             )

final =  alt.vconcat(detail,small_multiples).add_params(brush)

final.save('improved.html')
